In [18]:
import pandas as pd
import numpy as np
import pycountry
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.7 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681833 sha256=ec2234687a397bc99879390825431e4a2fd24460bfccd6500320ef9fdad85813
  Stored in directory: /Users/home/Library/Caches/pip/wheels/03/57/cc/290c5252ec97a6d78d36479a3c5e5ecc76318afcb241ad9dbe
Successfully built pycountry

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
#data = pd.read_csv("/Users/davidcastrejon/Downloads/candyhierarchy2017.csv", encoding="latin")
data = pd.read_csv("candyhierarchy2017.csv", encoding="latin")
data.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


# Column 1

In [ ]:
COLUMN_NAME = "Q1: GOING OUT?"
# convert to lowercase
data[COLUMN_NAME] = data[COLUMN_NAME].str.lower()
# Get count of yes, no, nan.
colOneValuesCount = data[COLUMN_NAME].astype(str).value_counts(dropna=False)
yesCountPercentage = colOneValuesCount.get("yes", 0) / (data[COLUMN_NAME].size - colOneValuesCount.get("nan", 0))
yesCountFill = round(yesCountPercentage * colOneValuesCount.get("nan", 0))
# shuffle the data frame 
data = data.sample(frac=1).reset_index(drop=True)
   # fill our nan values with yes according to the countFill
try:
    data[COLUMN_NAME].fillna("yes", limit=yesCountFill, inplace=True)
    data[COLUMN_NAME].fillna("no", inplace=True)
    data[COLUMN_NAME].unique()
except:
    pass
colOneValuesCount

# we can use regex validation for an extra layer of security (good for data integrity for large data sets)
 
# define reg pattern to use
regexPattern = r"^(yes|no)$" 

# parse through data to check if answers match regexPattern
def process_going_out(going_out):
    if re.match(regexPattern, going_out):
        return going_out
    else:
        return 'no'

data[COLUMN_NAME] = data[COLUMN_NAME].apply(process_going_out)
print(data[COLUMN_NAME].unique())


## Column 2

In [9]:
COLUMN_NAME = "Q2: GENDER"

data[COLUMN_NAME] = data[COLUMN_NAME].str.lower()
# data[COLUMN_NAME].unique() # checked unique values, only 'male', 'female', 'I'd rather not say', 'Other'
data[COLUMN_NAME].astype(str).fillna("Other", inplace=True)  
data[COLUMN_NAME].tail(2)


# we can use regex validation for an extra layer of security (good for data integrity for large data sets)

# define reg pattern to use
regexPattern = r"^(female|male|other|i'd rather not say)$" 

# parse through data to check if answers match regexPattern (as stated by slides, NIL values = 'other')
def process_gender(gender):
    gender_str = str(gender)
    match = re.match(regexPattern, gender_str)
    if match:
        return match.group()
    else:
        return 'other'
    
# apply column changes and print
data[COLUMN_NAME] = data[COLUMN_NAME].apply(process_gender)
print(data[COLUMN_NAME])


# Column 3

In [24]:
COLUMN_NAME = "Q3: AGE"
# Unique
#for the text data in the data frame that is age we can use reguaar expression to find the numbers and then convert them to numeric values
#
# 
"""
[nan, '44', '49', '40', '23', '53', '33', '43', '56', '64', '37',
       '59', '48', '54', '36', '45', '25', '34', '35', '38', '58', '50',
       '47', '16', '52', '63', '65', '41', '27', '31', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '26', '51',
       '70', '24', '18', '19', 'Old enough', '57', '60', '66', '12',
       'Many', '55', '72', '?', '21', '11', 'no', '9', '68', '20', '6',
       '10', '71', '90', '13', '45-55', '312', '99', '7', 'hahahahaha',
       '88', '39.4', '74', '102', 'older than dirt', '17', '15', '8',
       '75', '5u', 'Enough', 'See question 2', '24-50', '14', 'Over 50',
       '100', '76', '1000', 'sixty-nine', '46 Halloweens.', 'ancient',
       '77', 'OLD', 'old', '73', '70 1/2', '1', 'MY NAME JEFF', '4',
       '59 on the day after Halloween', 'old enough', 'your mom',
       'I can remember when Java was a cool new language', '60+']
"""

# Replace series as a numeric type rounded to the nearest whole number.
data[COLUMN_NAME] = pd.to_numeric(data[COLUMN_NAME], errors='coerce').round() # convert to numeric values & round

# remove columns which have values not in range 5-100
data[COLUMN_NAME] = data[COLUMN_NAME][(data[COLUMN_NAME] >= 5) & (data[COLUMN_NAME] <= 100)]  # filter to only ages of 5-100
# subsetcolumn label or sequence of labels, optional
data.dropna(subset=[COLUMN_NAME], inplace=True) # drop all rows with nan values
data[COLUMN_NAME]


# double check if there are any non-numeric values, this should be empty:
non_numeric_values = data[~data[COLUMN_NAME].apply(lambda x: np.isreal(x))]
message = "number of non-numeric values in column " + COLUMN_NAME + " (should be empty):"
print(message, non_numeric_values)

# fill nil values with the median of the age column (or mean if you want, i commented that out)
data[COLUMN_NAME].fillna(data[COLUMN_NAME].median(), inplace=True)
# data[COLUMN_NAME].fillna(data[COLUMN_NAME].mean(), inplace=True)

# double check to see if any nulls are left
message = "number of nulls in column " + COLUMN_NAME + " (should be 0):"
print (message, data[COLUMN_NAME].isna().sum())

number of non-numeric values in column Q3: AGE (should be empty): Empty DataFrame
Columns: [Internal ID, Q1: GOING OUT?, Q2: GENDER, Q3: AGE, Q4: COUNTRY, Q5: STATE, PROVINCE, COUNTY, ETC, Q6 | 100 Grand Bar, Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes), Q6 | Any full-sized candy bar, Q6 | Black Jacks, Q6 | Bonkers (the candy), Q6 | Bonkers (the board game), Q6 | Bottle Caps, Q6 | Box'o'Raisins, Q6 | Broken glow stick, Q6 | Butterfinger, Q6 | Cadbury Creme Eggs, Q6 | Candy Corn, Q6 | Candy that is clearly just the stuff given out for free at restaurants, Q6 | Caramellos, Q6 | Cash, or other forms of legal tender, Q6 | Chardonnay, Q6 | Chick-o-Sticks (we donÕt know what that is), Q6 | Chiclets, Q6 | Coffee Crisp, Q6 | Creepy Religious comics/Chick Tracts, Q6 | Dental paraphenalia, Q6 | Dots, Q6 | Dove Bars, Q6 | Fuzzy Peaches, Q6 | Generic Brand Acetaminophen, Q6 | Glow sticks, Q6 | Goo Goo Clusters, Q6 | Good N' Plenty, Q6 | Gum from baseball ca

# Column 4


In [25]:
# use the pycountry and autocorrect libraries
import pycountry
from autocorrect import Speller
COLUMN_NAME = "Q4: COUNTRY"

# initialize Speller obj from library
spell = Speller() 

# function to autocorrect country names
def autocorrectCountries(value):
    corrected_country = spell(value) # autocorrect name
    try:
        country = pycountry.countries.lookup(corrected_country) # look up country from pycountry
        if country:
            return country.name.lower() # if found, return the (lowercased) standard spelling
    except Exception as e:
        return str(np.nan)              # else return nan (not sure if there's something better?)

# apply to each country in column and retrieve all the unique names
data[COLUMN_NAME] = data[COLUMN_NAME].apply(autocorrectCountries) 
data[COLUMN_NAME].unique()

array(['korea, republic of', 'united states', 'iceland', 'nan', 'sweden',
       'canada', 'germany', 'netherlands', 'united kingdom',
       'switzerland', 'hong kong', 'france', 'singapore',
       'taiwan, province of china', 'mexico', 'japan', 'spain',
       'australia', 'south africa', 'ireland', 'china', 'indonesia',
       'finland', 'greece', 'denmark', 'costa rica'], dtype=object)

# Column 5

- ## Oppenheimer this column 💣

In [27]:
data.drop(columns=['Q5: STATE, PROVINCE, COUNTY, ETC'],inplace=True)

In [31]:
data.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90275757,yes,i'd rather not say,33.0,united states,00001,01000,00001,00100,00100,...,Bags of pennies,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(75, 26)"
1,90281870,yes,male,41.0,nan,00100,01000,00001,10000,10000,...,Future of american democracy,NaN,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(67, 12)"
2,90278266,no,female,49.0,nan,00001,00001,00001,00001,00100,...,diabetic candies,"Ultimately, any free sweetie is good sweetie.....",Blue and black,NaN,Friday,NaN,1.0,NaN,NaN,"(78, 30)"
3,90285679,no,female,44.0,united states,00001,01000,00001,01000,00100,...,Wax lips,Our local goodwill is selling wax lips current...,Blue and black,NaN,Sunday,NaN,1.0,NaN,NaN,"(76, 23)"
4,90281349,no,male,55.0,united states,00001,01000,00001,01000,01000,...,NaN,snickers #1,White and gold,NaN,Sunday,NaN,NaN,1.0,NaN,"(29, 67)"


# Columns 5 - 83

In [37]:
start_index = 5
end_index = 83

# Use list slicing to extract column names into a list
sequential_column_names = data.columns[start_index:end_index + 1].tolist()

for col in sequential_column_names:
    # Use the 'replace' method to map values to '001', '010', '100'
    data[col] = data[col].replace({'JOY': '00001', 'MEH': '00010', 'MEH': '00100', 'DESPAIR': '01000', np.nan: '10000'})
    data[col] = data[col].replace({'00001': 0, '00010': 1, '00100': 2, '01000': 3, '10000': 4})

# Show the first few rows of the data frame
data.head()


,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90275757,yes,i'd rather not say,33.0,united states,0,3,0,2,2,...,Bags of pennies,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(75, 26)"
1,90281870,yes,male,41.0,nan,2,3,0,4,4,...,Future of american democracy,NaN,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(67, 12)"
2,90278266,no,female,49.0,nan,0,0,0,0,2,...,diabetic candies,"Ultimately, any free sweetie is good sweetie.....",Blue and black,NaN,Friday,NaN,1.0,NaN,NaN,"(78, 30)"
3,90285679,no,female,44.0,united states,0,3,0,3,2,...,Wax lips,Our local goodwill is selling wax lips current...,Blue and black,NaN,Sunday,NaN,1.0,NaN,NaN,"(76, 23)"
4,90281349,no,male,55.0,united states,0,3,0,3,3,...,NaN,snickers #1,White and gold,NaN,Sunday,NaN,NaN,1.0,NaN,"(29, 67)"


In [36]:
data['Q3: AGE'].median()

41.0